In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from sklearn.model_selection import train_test_split

In [ ]:
# Load the Ionosphere dataset
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/ionosphere/ionosphere.data'
data = pd.read_csv(url, header=None)

X = data.iloc[:, :-1].values  # Input features
y = data.iloc[:, -1].replace({'g': 1, 'b': 0}).values  # Class labels

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Autoencoder without added noise and weight regularization
input_dim = X_train.shape[1]  # Input dimension

# Define the autoencoder architecture
input_layer = Input(shape=(input_dim,))
encoded = Dense(16, activation='relu')(input_layer)  # Hidden layer
decoded = Dense(input_dim, activation='sigmoid')(encoded)  # Output layer

autoencoder = Model(input_layer, decoded)

# Compile the model
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

In [ ]:
# Train the autoencoder
autoencoder.fit(X_train, X_train, epochs=50, batch_size=16, validation_data=(X_test, X_test))

In [ ]:
# Add Gaussian noise to the training data
noise_factor = 0.2
X_train_noisy = X_train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=X_train.shape)

# Clip the noisy data to ensure values are between 0 and 1
X_train_noisy = np.clip(X_train_noisy, 0., 1.)

# Autoencoder with added Gaussian noise and no weight regularization
input_dim = X_train.shape[1]  # Input dimension

# Define the autoencoder architecture
input_layer = Input(shape=(input_dim,))
encoded = Dense(16, activation='relu')(input_layer)  # Hidden layer
decoded = Dense(input_dim, activation='sigmoid')(encoded)  # Output layer

autoencoder_with_noise = Model(input_layer, decoded)

# Compile the model
autoencoder_with_noise.compile(optimizer='adam', loss='binary_crossentropy')

In [ ]:
# Train the autoencoder
autoencoder_with_noise.fit(X_train_noisy, X_train, epochs=50, batch_size=16, validation_data=(X_test, X_test))

In [ ]:
# Reconstruct inputs using the trained autoencoders
reconstructed = autoencoder.predict(X_test)
reconstructed_with_noise = autoencoder_with_noise.predict(X_test)

In [ ]:
# Calculate MSE between original inputs and reconstructed inputs
mse = np.mean(np.square(X_test - reconstructed))
mse_with_noise = np.mean(np.square(X_test - reconstructed_with_noise))

print("MSE - Case (a):", mse)
print("MSE - Case (b):", mse_with_noise)